# LOAD DATA


In [3]:
import pandas as pd

In [4]:
matches = pd.read_csv('../data/matches.csv',index_col=0)
print(matches.head())



         date   time            comp        round  day venue result   gf   ga  \
1  2021-08-15  16:30  Premier League  Matchweek 1  Sun  Away      L  0.0  1.0   
2  2021-08-21  15:00  Premier League  Matchweek 2  Sat  Home      W  5.0  0.0   
3  2021-08-28  12:30  Premier League  Matchweek 3  Sat  Home      W  5.0  0.0   
4  2021-09-11  15:00  Premier League  Matchweek 4  Sat  Away      W  1.0  0.0   
6  2021-09-18  15:00  Premier League  Matchweek 5  Sat  Home      D  0.0  0.0   

         opponent  ...  match report  notes    sh   sot  dist   fk   pk pkatt  \
1       Tottenham  ...  Match Report    NaN  18.0   4.0  16.9  1.0  0.0   0.0   
2    Norwich City  ...  Match Report    NaN  16.0   4.0  17.3  1.0  0.0   0.0   
3         Arsenal  ...  Match Report    NaN  25.0  10.0  14.3  0.0  0.0   0.0   
4  Leicester City  ...  Match Report    NaN  25.0   8.0  14.0  0.0  0.0   0.0   
6     Southampton  ...  Match Report    NaN  16.0   1.0  15.7  1.0  0.0   0.0   

   season             team

In [5]:
print(matches.shape)

(1389, 27)


In [6]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk              float64
pkatt           float64
season            int64
team             object
dtype: object

In [7]:
matches['date'] = pd.to_datetime(matches['date'])
matches.dtypes

date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                     float64
ga                     float64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                  float64
sh                     float64
sot                    float64
dist                   float64
fk                     float64
pk                     float64
pkatt                  float64
season                   int64
team                    object
dtype: object

In [8]:
matches['venue_code'] = matches['venue'].astype('category').cat.codes

In [9]:
matches['opp_code'] = matches['opponent'].astype('category').cat.codes


In [10]:
matches['hour'] = matches['time'].str.replace(':.+', '', regex=True).astype("int")


In [11]:
matches['day_code'] = matches['date'].dt.dayofweek


In [12]:
matches["target"] = (matches["result"] == 'W').astype("int")

In [13]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)


In [14]:
train = matches[matches['date'] < '2022-01-01']

In [15]:
test = matches[matches['date'] >= '2022-01-01']

In [16]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [17]:
rf.fit(train[predictors], train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [18]:
preds = rf.predict(test[predictors])

In [19]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(test["target"], preds)

acc

0.6134751773049646

In [20]:
combined = pd.DataFrame(dict(actual=test['target'], prediction=preds))
pd.crosstab(index=combined["actual"], columns=combined['prediction'])

prediction,0,1
actual,,
0,144,31
1,78,29


In [21]:
from sklearn.metrics import precision_score

precision_score(test['target'], preds)

0.48333333333333334

In [22]:
grouped_matches = matches.groupby('team')


In [23]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

    

In [24]:
cols = ['gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt']
new_cols = [f"{c}_rolling" for c in cols]

In [25]:
group = grouped_matches.get_group('Manchester City')
rolling_averages(group, cols, new_cols)


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
5,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Home,W,1.0,0.0,Arsenal,...,5,1,2.000000,2.333333,17.333333,4.666667,18.900000,1.333333,0.333333,0.333333
7,2020-10-24,12:30,Premier League,Matchweek 6,Sat,Away,D,1.0,1.0,West Ham,...,5,0,1.333333,2.000000,17.333333,3.666667,17.733333,0.666667,0.000000,0.000000
9,2020-10-31,12:30,Premier League,Matchweek 7,Sat,Away,W,1.0,0.0,Sheffield Utd,...,5,1,1.000000,0.666667,16.666667,4.333333,18.233333,0.666667,0.000000,0.000000
11,2020-11-08,16:30,Premier League,Matchweek 8,Sun,Home,D,1.0,1.0,Liverpool,...,6,0,1.000000,0.333333,14.333333,6.666667,18.466667,1.000000,0.000000,0.000000
12,2020-11-21,17:30,Premier League,Matchweek 9,Sat,Away,L,0.0,2.0,Tottenham,...,5,0,1.000000,0.666667,12.000000,5.666667,19.366667,1.000000,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2022-03-14,20:00,Premier League,Matchweek 29,Mon,Away,D,0.0,0.0,Crystal Palace,...,0,0,2.333333,1.333333,19.000000,7.000000,15.366667,0.333333,0.333333,0.333333
44,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Away,W,2.0,0.0,Burnley,...,5,1,1.666667,0.333333,18.333333,7.333333,16.000000,0.333333,0.000000,0.000000
46,2022-04-10,16:30,Premier League,Matchweek 32,Sun,Home,D,2.0,2.0,Liverpool,...,6,0,2.000000,0.333333,20.000000,6.666667,16.133333,0.333333,0.000000,0.000000
49,2022-04-20,20:00,Premier League,Matchweek 30,Wed,Home,W,3.0,0.0,Brighton,...,2,1,1.333333,0.666667,15.666667,4.666667,16.700000,0.333333,0.000000,0.000000


In [26]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

C:\Users\Othman SALAHI\AppData\Local\Temp\ipykernel_17520\4052147919.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [27]:
matches_rolling = matches_rolling.droplevel("team")


In [28]:
matches_rolling.index = range(matches_rolling.shape[0])


In [29]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [30]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)


In [31]:
combined

,actual,predicted
55,0,0
56,1,0
57,1,0
58,1,1
59,1,1
...,...,...
1312,1,0
1313,0,0
1314,1,0
1315,0,0


In [32]:
combined = combined.merge(matches_rolling[['date', 'team', 'opponent', 'result']], left_index=True, right_index=True)

In [33]:
combined

,actual,predicted,date,team,opponent,result
55,0,0,2022-01-23,Arsenal,Burnley,D
56,1,0,2022-02-10,Arsenal,Wolves,W
57,1,0,2022-02-19,Arsenal,Brentford,W
58,1,1,2022-02-24,Arsenal,Wolves,W
59,1,1,2022-03-06,Arsenal,Watford,W
...,...,...,...,...,...,...
1312,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W
1313,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L
1314,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W
1315,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L


In [34]:
import numpy as np
combined['result_predicted'] = np.where(combined['predicted'] == 1, 'W', 'L or D')
combined

,actual,predicted,date,team,opponent,result,result_predicted
55,0,0,2022-01-23,Arsenal,Burnley,D,L or D
56,1,0,2022-02-10,Arsenal,Wolves,W,L or D
57,1,0,2022-02-19,Arsenal,Brentford,W,L or D
58,1,1,2022-02-24,Arsenal,Wolves,W,W
59,1,1,2022-03-06,Arsenal,Watford,W,W
...,...,...,...,...,...,...,...
1312,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,L or D
1313,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L,L or D
1314,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,L or D
1315,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,L or D


In [35]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", 
              "Manchester United": "Manchester Utd", 
              "Newcastle United": "Newcastle Utd", 
              "Tottenham Hotspur": "Tottenham", 
              "West Ham United": "West Ham", 
              "Wolverhampton Wanderers": "Wolves"} 
mapping = MissingDict(**map_values)

In [36]:
mapping['Liverpool']

'Liverpool'

In [37]:
combined["new_team"] = combined["team"].map(mapping)

In [42]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [43]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,result_predicted_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,result_predicted_y,new_team_y
0,0,0,2022-01-23,Arsenal,Burnley,D,L or D,Arsenal,0,0,Burnley,Arsenal,D,L or D,Burnley
1,1,0,2022-02-10,Arsenal,Wolves,W,L or D,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,L or D,Wolves
2,1,0,2022-02-19,Arsenal,Brentford,W,L or D,Arsenal,0,0,Brentford,Arsenal,L,L or D,Brentford
3,1,1,2022-02-24,Arsenal,Wolves,W,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,L or D,Wolves
4,1,1,2022-03-06,Arsenal,Watford,W,W,Arsenal,0,0,Watford,Arsenal,L,L or D,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,L or D,Wolves,0,0,Everton,Wolves,L,L or D,Everton
258,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L,L or D,Wolves,1,0,Leeds United,Wolves,W,L or D,Leeds United
259,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,L or D,Wolves,0,0,Aston Villa,Wolves,L,L or D,Aston Villa
260,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,L or D,Wolves,1,0,Newcastle United,Wolves,W,L or D,Newcastle Utd


In [41]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()

actual_x
1    27
0    13
Name: count, dtype: int64